# Process Products from Ingredient Rows CSV

This notebook processes the large `ingredient_rows.csv` file (50MB+) to:
1. Read ingredient-product relationships
2. Group by product (using product_url as unique identifier)
3. Create a list of ingredient_ids for each product
4. Generate SQL to create a `products_dim` table in PostgreSQL

## 1. Import Required Libraries

In [1]:
import pandas as pd
import os
from pathlib import Path
import hashlib
import re

# Paths
BASE_DIR = Path(r"D:\IPSSI\skinintel")
INGREDIENT_ROWS_CSV = BASE_DIR / "ingredient_rows.csv"
INGREDIENTS_DIM_CACHE = BASE_DIR / "ingredients_dim_cache.csv"
OUTPUT_SQL = BASE_DIR / "webapp" / "sql" / "seed_products.sql"

print(f"Ingredient rows CSV: {INGREDIENT_ROWS_CSV}")
print(f"Ingredients dim cache: {INGREDIENTS_DIM_CACHE}")
print(f"Output SQL: {OUTPUT_SQL}")

Ingredient rows CSV: D:\IPSSI\skinintel\ingredient_rows.csv
Ingredients dim cache: D:\IPSSI\skinintel\ingredients_dim_cache.csv
Output SQL: D:\IPSSI\skinintel\webapp\sql\seed_products.sql


## 2. Load the Ingredients Dimension Cache

First, load the ingredients dimension cache to get all known ingredient IDs.

In [2]:
# Load ingredients dimension cache
ingredients_dim = pd.read_csv(INGREDIENTS_DIM_CACHE)

print(f"Loaded {len(ingredients_dim)} ingredients from dimension cache")
print(f"Columns: {ingredients_dim.columns.tolist()}")
ingredients_dim.head()

Loaded 2648 ingredients from dimension cache
Columns: ['ingredient_id', 'ingredient', 'ingredient_url', 'functions', 'concerns']


,ingredient_id,ingredient,ingredient_url,functions,concerns
0,ing_09ef0af7963abef4,ALOE BARBADENSIS (ALOE VERA) LEAF JUICE Data A...,https://www.ewg.org/skindeep/ingredients/70027...,[],"[""Use restrictions (low)""]"
1,ing_c26aded2ba3279f3,WATER Data Availability: Robust,https://www.ewg.org/skindeep/ingredients/70694...,"[""solvent""]",[]
2,ing_4cbcae840c7167dd,HAMAMELIS VIRGINIANA (WITCH HAZEL) WATER Data ...,https://www.ewg.org/skindeep/ingredients/70277...,"[""drug astringent - skin protectant drugs"", ""s...","[""Cancer (low)"", ""Developmental/reproductive t..."
3,ing_cc44df068045a413,SALIX NIGRA (BLACK WILLOW) BARK EXTRACT Data A...,https://www.ewg.org/skindeep/ingredients/70575...,"[""skin protecting"", ""not reported""]","[""Cancer (low)"", ""Developmental/reproductive t..."
4,ing_2f5d30dfbf3e01ec,LEUCONOSTOC/ RADISH ROOT FERMENT FILTRATE Data...,https://www.ewg.org/skindeep/ingredients/72960...,"[""antidandruff"", ""antimicrobial""]","[""Contamination concerns (Didecyldimethylammon..."


## 2.1 Create Mapping from Original IDs to Simplified IDs

The enriched ingredients table uses simplified IDs like `aloe_vera` instead of the original `ing_xxxxx` format.
We need to create a mapping using the same logic as the enrichment script.

In [10]:
def clean_ingredient_name(raw_name: str) -> tuple:
    """
    Extract clean ingredient name from EWG format (same logic as enrich_cache_to_sql.py)
    Input: "ALOE BARBADENSIS (ALOE VERA) LEAF JUICE Data Availability: Good"
    Output: ("aloe_vera", "Aloe Vera Leaf Juice")
    """
    # Remove "Data Availability: ..." suffix
    name = re.sub(r'\s*Data Availability:.*$', '', raw_name, flags=re.IGNORECASE)
    
    # Extract common name from parentheses if exists
    match = re.search(r'\(([^)]+)\)', name)
    if match:
        common_name = match.group(1)
    else:
        common_name = name
    
    # Create display name (title case)
    display_name = name.title()
    
    # Create snake_case ID
    clean = re.sub(r'[^a-zA-Z0-9\s]', '', common_name)
    simplified_id = clean.lower().strip().replace(' ', '_')
    
    # Limit length to match the enrichment script
    if len(simplified_id) > 30:
        simplified_id = simplified_id[:30]
    
    return simplified_id, display_name

# Create mapping: original_id (ing_xxx) -> simplified_id (aloe_vera)
original_to_simplified = {}
simplified_to_original = {}

for _, row in ingredients_dim.iterrows():
    original_id = row['ingredient_id']  # ing_xxxxx
    ingredient_name = row['ingredient']  # ALOE BARBADENSIS...
    
    simplified_id, _ = clean_ingredient_name(ingredient_name)
    
    original_to_simplified[original_id] = simplified_id
    simplified_to_original[simplified_id] = original_id

print(f"Created mapping for {len(original_to_simplified)} ingredients")
print(f"\nSample mappings:")
for i, (orig, simp) in enumerate(list(original_to_simplified.items())[:5]):
    print(f"  {orig} -> {simp}")

Created mapping for 2648 ingredients

Sample mappings:
  ing_09ef0af7963abef4 -> aloe_vera
  ing_c26aded2ba3279f3 -> water
  ing_4cbcae840c7167dd -> witch_hazel
  ing_cc44df068045a413 -> black_willow
  ing_2f5d30dfbf3e01ec -> leuconostoc_radish_root_fermen


## 3. Load the Ingredient Rows CSV (Chunked Reading)

The file is 50MB+, so we'll read it in chunks and process incrementally.

In [3]:
# Read ingredient_rows.csv in chunks to handle the large file
CHUNK_SIZE = 100_000

# Dictionary to store product -> list of ingredient_ids
product_data = {}

# Read and process in chunks
chunk_count = 0
for chunk in pd.read_csv(INGREDIENT_ROWS_CSV, chunksize=CHUNK_SIZE):
    chunk_count += 1
    print(f"Processing chunk {chunk_count} ({len(chunk)} rows)...")
    
    for _, row in chunk.iterrows():
        product_url = row['product_url']
        product_name = row['product']
        category = row['category']
        company = row['company']
        ingredient_id = row['ingredient_id']
        
        if product_url not in product_data:
            product_data[product_url] = {
                'product_name': product_name,
                'category': category,
                'company': company,
                'ingredient_ids': set()  # Use set to avoid duplicates
            }
        
        # Add ingredient_id to the set
        if pd.notna(ingredient_id):
            product_data[product_url]['ingredient_ids'].add(ingredient_id)

print(f"\nProcessed {chunk_count} chunks")
print(f"Found {len(product_data)} unique products")

Processing chunk 1 (100000 rows)...
Processing chunk 2 (99973 rows)...

Processed 2 chunks
Found 9023 unique products


## 4. Create Products DataFrame

Convert the dictionary to a DataFrame with proper structure.

In [4]:
# Create a list of product records
products_list = []

for product_url, data in product_data.items():
    # Generate a product_id from the product_url hash
    product_id = 'prod_' + hashlib.md5(product_url.encode()).hexdigest()[:16]
    
    # Convert set to sorted list for consistent ordering
    ingredient_ids = sorted(list(data['ingredient_ids']))
    
    products_list.append({
        'product_id': product_id,
        'product_name': data['product_name'],
        'product_url': product_url,
        'category': data['category'],
        'company': data['company'],
        'ingredient_ids': ingredient_ids,
        'ingredient_count': len(ingredient_ids)
    })

# Create DataFrame
products_df = pd.DataFrame(products_list)

# Sort by ingredient count descending to see products with most ingredients
products_df = products_df.sort_values('ingredient_count', ascending=False).reset_index(drop=True)

print(f"Created DataFrame with {len(products_df)} products")
print(f"\nColumns: {products_df.columns.tolist()}")
print(f"\nIngredient count statistics:")
print(products_df['ingredient_count'].describe())

products_df.head(10)

Created DataFrame with 9023 products

Columns: ['product_id', 'product_name', 'product_url', 'category', 'company', 'ingredient_ids', 'ingredient_count']

Ingredient count statistics:
count    9023.000000
mean       21.612546
std        12.611961
min         1.000000
25%        13.000000
50%        20.000000
75%        29.000000
max       149.000000
Name: ingredient_count, dtype: float64


,product_id,product_name,product_url,category,company,ingredient_ids,ingredient_count
0,prod_0b07318b504b753f,A.H.C,https://www.ewg.org/skindeep/products/947199-A...,Around-Eye_Cream,A.H.C,"[ing_028c4e4a5e881824, ing_044611a7c3c5916f, i...",149
1,prod_2f7a91a3912cc5ec,La Mer,https://www.ewg.org/skindeep/products/3915975-...,Around-Eye_Cream,La Mer,"[ing_028bdb74dc79be94, ing_04626122c86914d9, i...",115
2,prod_2c85098c89e7d1a1,It Cosmetics,https://www.ewg.org/skindeep/products/965721-I...,Anti-Aging,It Cosmetics,"[ing_095eb1b0e1e32618, ing_0e4799a2dcecda4c, i...",102
3,prod_b853289e04279e55,La Mer,https://www.ewg.org/skindeep/products/5602681-...,Anti-Aging,La Mer,"[ing_028bdb74dc79be94, ing_04626122c86914d9, i...",99
4,prod_a7d7b48dfd5346f2,It Cosmetics,https://www.ewg.org/skindeep/products/968497-I...,Anti-Aging,It Cosmetics,"[ing_05d84c462eaddde6, ing_081fabb37bf7a435, i...",98
5,prod_27863252e126da77,La Mer,https://www.ewg.org/skindeep/products/951147-L...,Around-Eye_Cream,La Mer,"[ing_028bdb74dc79be94, ing_04be797ce36730fd, i...",88
6,prod_fa431acc8da96f78,Brickell Men's Products,https://www.ewg.org/skindeep/products/966014-B...,Around-Eye_Cream,Brickell Men's Products,"[ing_04d743872a0820fe, ing_0521fa37da131683, i...",86
7,prod_0503d3ef7bc35687,Darphin,https://www.ewg.org/skindeep/products/7540510-...,Anti-Aging,Darphin,"[ing_028bdb74dc79be94, ing_0521fa37da131683, i...",81
8,prod_bb57ff4f12283352,La Prairie,https://www.ewg.org/skindeep/products/5086842-...,Around-Eye_Cream,La Prairie,"[ing_06337debe8fb7255, ing_0c6ff90fffa4eae6, i...",79
9,prod_d5500f822e8f411c,Image Skincare,https://www.ewg.org/skindeep/products/1061163-...,Anti-Aging,Image Skincare,"[ing_029aaf9c726f7da2, ing_082af2b0ccf22fdb, i...",79


## 5. Validate Ingredient IDs

Check how many of our ingredient_ids match the ingredients_dim_cache.

In [5]:
# Get set of all known ingredient_ids from dimension cache
known_ingredient_ids = set(ingredients_dim['ingredient_id'].tolist())

# Get all unique ingredient_ids from products
all_product_ingredient_ids = set()
for ids in products_df['ingredient_ids']:
    all_product_ingredient_ids.update(ids)

# Find matches and mismatches
matched_ids = all_product_ingredient_ids.intersection(known_ingredient_ids)
unmatched_ids = all_product_ingredient_ids - known_ingredient_ids

print(f"Total unique ingredient_ids in products: {len(all_product_ingredient_ids)}")
print(f"Matched with ingredients_dim_cache: {len(matched_ids)} ({100*len(matched_ids)/len(all_product_ingredient_ids):.1f}%)")
print(f"Unmatched (not in dim cache): {len(unmatched_ids)} ({100*len(unmatched_ids)/len(all_product_ingredient_ids):.1f}%)")

# Show some unmatched examples
if unmatched_ids:
    print(f"\nSample unmatched ingredient_ids: {list(unmatched_ids)[:5]}")

Total unique ingredient_ids in products: 7043
Matched with ingredients_dim_cache: 2648 (37.6%)
Unmatched (not in dim cache): 4395 (62.4%)

Sample unmatched ingredient_ids: ['ing_513b67b1fd5d7f05', 'ing_2579591a6dc2b679', 'ing_6638767a1a3c171e', 'ing_5b35119e2bdf3ed0', 'ing_2ae0846cff2c1673']


## 6. Filter Products to Only Include Known Ingredients

Keep only ingredient_ids that exist in our database.

In [11]:
# Filter and convert ingredient_ids to simplified format
def filter_and_convert_ingredients(ingredient_ids):
    """
    Convert original ing_xxx IDs to simplified IDs (aloe_vera) 
    and filter to only those in our dimension cache
    """
    simplified_ids = []
    for orig_id in ingredient_ids:
        if orig_id in original_to_simplified:
            simplified_id = original_to_simplified[orig_id]
            simplified_ids.append(simplified_id)
    return simplified_ids

products_df['ingredient_ids_simplified'] = products_df['ingredient_ids'].apply(filter_and_convert_ingredients)
products_df['ingredient_count_simplified'] = products_df['ingredient_ids_simplified'].apply(len)

# Only keep products that have at least 1 known ingredient
products_with_ingredients = products_df[products_df['ingredient_count_simplified'] > 0].copy()

print(f"Products with at least 1 known ingredient: {len(products_with_ingredients)} / {len(products_df)}")
print(f"\nSimplified ingredient count statistics:")
print(products_with_ingredients['ingredient_count_simplified'].describe())

# Show sample conversion
print(f"\nSample ID conversions for first product:")
sample_product = products_with_ingredients.iloc[0]
print(f"  Product: {sample_product['product_name']}")
print(f"  Original IDs: {sample_product['ingredient_ids'][:3]}...")
print(f"  Simplified IDs: {sample_product['ingredient_ids_simplified'][:3]}...")

Products with at least 1 known ingredient: 8264 / 9023

Simplified ingredient count statistics:
count    8264.000000
mean       20.717328
std        11.685466
min         1.000000
25%        12.000000
50%        19.000000
75%        28.000000
max       119.000000
Name: ingredient_count_simplified, dtype: float64

Sample ID conversions for first product:
  Product: A.H.C
  Original IDs: ['ing_028c4e4a5e881824', 'ing_044611a7c3c5916f', 'ing_081fabb37bf7a435']...
  Simplified IDs: ['magnesium_pca', 'dipropylene_glycol', 'common_yarrow']...


## 7. Generate SQL for Products Table

Create SQL statements to:
1. Create the `products_dim` table
2. Insert all products with their ingredient lists (as PostgreSQL arrays)

In [12]:
def escape_sql_string(s):
    """Escape single quotes for SQL"""
    if pd.isna(s):
        return ''
    return str(s).replace("'", "''")

def format_pg_array(items):
    """Format a list as PostgreSQL array literal"""
    if not items:
        return "ARRAY[]::VARCHAR[]"
    escaped = [f"'{escape_sql_string(item)}'" for item in items]
    return f"ARRAY[{', '.join(escaped)}]"

# Generate SQL
sql_lines = []

# Header
sql_lines.append("-- Products Dimension Table")
sql_lines.append("-- Generated from ingredient_rows.csv")
sql_lines.append(f"-- Total products: {len(products_with_ingredients)}")
sql_lines.append("-- ingredient_ids use simplified format (e.g., 'aloe_vera') matching active_ingredients_database")
sql_lines.append("")

# Create table
sql_lines.append("""-- Create products_dim table
CREATE TABLE IF NOT EXISTS products_dim (
    product_id VARCHAR(32) PRIMARY KEY,
    product_name TEXT NOT NULL,
    product_url TEXT,
    category VARCHAR(100),
    company VARCHAR(255),
    ingredient_ids VARCHAR(50)[],
    ingredient_count INTEGER,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")

# Truncate for fresh insert
sql_lines.append("-- Clear existing data")
sql_lines.append("TRUNCATE TABLE products_dim;")
sql_lines.append("")

# Insert statements - using simplified IDs that match the active_ingredients_database
sql_lines.append("-- Insert products (ingredient_ids match active_ingredients_database.ingredient_id)")
for _, row in products_with_ingredients.iterrows():
    product_id = escape_sql_string(row['product_id'])
    product_name = escape_sql_string(row['product_name'])
    product_url = escape_sql_string(row['product_url'])
    category = escape_sql_string(row['category'])
    company = escape_sql_string(row['company'])
    ingredient_ids = row['ingredient_ids_simplified']  # Use simplified IDs!
    ingredient_count = row['ingredient_count_simplified']
    
    pg_array = format_pg_array(ingredient_ids)
    
    sql = f"""INSERT INTO products_dim (product_id, product_name, product_url, category, company, ingredient_ids, ingredient_count)
VALUES ('{product_id}', '{product_name}', '{product_url}', '{category}', '{company}', {pg_array}, {ingredient_count});"""
    sql_lines.append(sql)

# Create index for ingredient search
sql_lines.append("")
sql_lines.append("-- Create GIN index for efficient ingredient array searches")
sql_lines.append("CREATE INDEX IF NOT EXISTS idx_products_ingredients ON products_dim USING GIN (ingredient_ids);")
sql_lines.append("CREATE INDEX IF NOT EXISTS idx_products_category ON products_dim (category);")
sql_lines.append("CREATE INDEX IF NOT EXISTS idx_products_company ON products_dim (company);")

sql_content = '\n'.join(sql_lines)
print(f"Generated {len(sql_lines)} lines of SQL")
print(f"\nFirst 60 lines:")
print('\n'.join(sql_lines[:60]))

Generated 8279 lines of SQL

First 60 lines:
-- Products Dimension Table
-- Generated from ingredient_rows.csv
-- Total products: 8264
-- ingredient_ids use simplified format (e.g., 'aloe_vera') matching active_ingredients_database

-- Create products_dim table
CREATE TABLE IF NOT EXISTS products_dim (
    product_id VARCHAR(32) PRIMARY KEY,
    product_name TEXT NOT NULL,
    product_url TEXT,
    category VARCHAR(100),
    company VARCHAR(255),
    ingredient_ids VARCHAR(50)[],
    ingredient_count INTEGER,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Clear existing data
TRUNCATE TABLE products_dim;

-- Insert products (ingredient_ids match active_ingredients_database.ingredient_id)
INSERT INTO products_dim (product_id, product_name, product_url, category, company, ingredient_ids, ingredient_count)
VALUES ('prod_0b07318b504b753f', 'A.H.C', 'https://www.ewg.org/skindeep/products/947199-AHC_AHC_Essential_Real_Eye_Cream_for_Face_2020_formulation/', 'Around-Eye_Cream', 'A.H.

## 8. Save SQL File

In [13]:
# Save SQL to file
with open(OUTPUT_SQL, 'w', encoding='utf-8') as f:
    f.write(sql_content)

print(f"SQL file saved to: {OUTPUT_SQL}")
print(f"File size: {OUTPUT_SQL.stat().st_size / 1024 / 1024:.2f} MB")

SQL file saved to: D:\IPSSI\skinintel\webapp\sql\seed_products.sql
File size: 5.41 MB


## 9. Push to Database (Optional)

Run this cell to directly load the data into PostgreSQL.

In [16]:
import psycopg2
from psycopg2.extras import execute_values

# Database connection settings (same as docker-compose)
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'airflow',
    'user': 'airflow',
    'password': 'airflow'
}

def push_to_database():
    """Push products data to PostgreSQL database with simplified ingredient IDs"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    try:
        # Create table with larger varchar for simplified IDs
        cur.execute("""
            CREATE TABLE IF NOT EXISTS products_dim (
                product_id VARCHAR(32) PRIMARY KEY,
                product_name TEXT NOT NULL,
                product_url TEXT,
                category VARCHAR(100),
                company VARCHAR(255),
                ingredient_ids VARCHAR(50)[],
                ingredient_count INTEGER,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        
        # Truncate existing data
        cur.execute("TRUNCATE TABLE products_dim;")
        
        # Prepare data for bulk insert - using SIMPLIFIED IDs
        data = []
        for _, row in products_with_ingredients.iterrows():
            data.append((
                row['product_id'],
                row['product_name'],
                row['product_url'],
                row['category'],
                row['company'],
                row['ingredient_ids_simplified'],  # Use simplified IDs!
                row['ingredient_count_simplified']
            ))
        
        # Bulk insert using execute_values for speed
        insert_sql = """
            INSERT INTO products_dim 
            (product_id, product_name, product_url, category, company, ingredient_ids, ingredient_count)
            VALUES %s
        """
        execute_values(cur, insert_sql, data, page_size=1000)
        
        # Create indexes
        cur.execute("CREATE INDEX IF NOT EXISTS idx_products_ingredients ON products_dim USING GIN (ingredient_ids);")
        cur.execute("CREATE INDEX IF NOT EXISTS idx_products_category ON products_dim (category);")
        cur.execute("CREATE INDEX IF NOT EXISTS idx_products_company ON products_dim (company);")
        
        conn.commit()
        
        # Verify
        cur.execute("SELECT COUNT(*) FROM products_dim;")
        count = cur.fetchone()[0]
        print(f"✅ Successfully inserted {count} products into database!")
        
        # Show sample to verify simplified IDs
        cur.execute("SELECT product_name, ingredient_ids[1:3], ingredient_count FROM products_dim LIMIT 3;")
        print("\nSample products (showing first 3 ingredients):")
        for row in cur.fetchall():
            print(f"  - {row[0]}: {row[1]} ({row[2]} total)")
        
        # Test join with active_ingredients_database
        cur.execute("""
            SELECT p.product_name, COUNT(DISTINCT i.ingredient_id) as matched_count
            FROM products_dim p, 
                 LATERAL unnest(p.ingredient_ids) AS ing_id
            JOIN active_ingredients_database i ON i.ingredient_id = ing_id
            GROUP BY p.product_name
            ORDER BY matched_count DESC
            LIMIT 5;
        """)
        print("\nTop 5 products by matched ingredients in DB:")
        for row in cur.fetchall():
            print(f"  - {row[0]}: {row[1]} ingredients found in active_ingredients_database")
            
    except Exception as e:
        conn.rollback()
        print(f"❌ Error: {e}")
        raise
    finally:
        cur.close()
        conn.close()

# Run the database push
push_to_database()

✅ Successfully inserted 8264 products into database!

Sample products (showing first 3 ingredients):
  - A.H.C: ['magnesium_pca', 'dipropylene_glycol', 'common_yarrow'] (119 total)
  - La Mer: ['species_unspecified', 'micrococcus_lysate', 'hydroxycitronellal'] (97 total)
  - It Cosmetics: ['glyceryl_monostearate', 'lecithin', 'chrysin'] (89 total)

Top 5 products by matched ingredients in DB:
  - Clinique: 303 ingredients found in active_ingredients_database
  - Bliss: 294 ingredients found in active_ingredients_database
  - Pixi: 290 ingredients found in active_ingredients_database
  - BABOR: 285 ingredients found in active_ingredients_database
  - Neutrogena: 279 ingredients found in active_ingredients_database


## 10. Test: Find Products by Ingredients

This shows how to query products that contain specific ingredients (what we'll use to match scanned products).

In [ ]:
def find_products_by_ingredients(scanned_ingredient_ids, top_n=5):
    """
    Find products that best match a set of scanned ingredient IDs.
    Uses Jaccard similarity to rank matches.
    
    Args:
        scanned_ingredient_ids: List of ingredient_ids detected from OCR
        top_n: Number of top matches to return
    
    Returns:
        List of (product_name, product_url, similarity_score, matched_ingredients)
    """
    scanned_set = set(scanned_ingredient_ids)
    
    results = []
    for _, row in products_with_ingredients.iterrows():
        product_ingredients = set(row['ingredient_ids_filtered'])
        
        # Calculate Jaccard similarity
        intersection = len(scanned_set.intersection(product_ingredients))
        union = len(scanned_set.union(product_ingredients))
        
        if union > 0 and intersection > 0:
            similarity = intersection / union
            # Also calculate match percentage (what % of scanned ingredients are in this product)
            match_pct = intersection / len(scanned_set) if scanned_set else 0
            
            results.append({
                'product_name': row['product_name'],
                'product_url': row['product_url'],
                'category': row['category'],
                'company': row['company'],
                'similarity': similarity,
                'match_pct': match_pct,
                'matched_count': intersection,
                'total_scanned': len(scanned_set),
                'total_product': len(product_ingredients)
            })
    
    # Sort by match percentage then similarity
    results.sort(key=lambda x: (x['match_pct'], x['similarity']), reverse=True)
    return results[:top_n]

# Example: Test with some sample ingredient IDs from our data
sample_ingredients = list(known_ingredient_ids)[:5]
print(f"Testing with sample ingredients: {sample_ingredients}")
print()

matches = find_products_by_ingredients(sample_ingredients, top_n=3)
for i, match in enumerate(matches, 1):
    print(f"{i}. {match['product_name']}")
    print(f"   Company: {match['company']}")
    print(f"   Category: {match['category']}")
    print(f"   Match: {match['matched_count']}/{match['total_scanned']} ingredients ({match['match_pct']*100:.1f}%)")
    print(f"   Similarity: {match['similarity']*100:.1f}%")
    print()

## Summary

This notebook:
1. ✅ Loaded `ingredient_rows.csv` in chunks (handles 50MB+ files)
2. ✅ Grouped rows by product (using `product_url` as unique key)
3. ✅ Created a list of `ingredient_ids` for each product
4. ✅ Filtered to only include ingredients in our `ingredients_dim_cache`
5. ✅ Generated SQL to create `products_dim` table
6. ✅ Saved SQL to `webapp/sql/seed_products.sql`
7. ✅ Provided function to push directly to PostgreSQL
8. ✅ Demonstrated product matching by ingredients

**Next Steps:**
- Run the SQL file or `push_to_database()` to populate the database
- Update the webapp to use `products_dim` for matching scanned ingredients to real products